In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model
import pickle # saving and loading trained model
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# My Data load
multi_data = pd.read_csv('/content/drive/MyDrive/multi_data.csv')
multi_data.drop(multi_data.columns[0],axis=1,inplace=True)
le2_classes_ = np.load('/content/drive/MyDrive/le2_classes.npy',allow_pickle=True)

In [8]:
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense # importing dense layer

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained modelS

import keras
import tensorflow as tf

In [5]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test = train_test_split(multi_data, test_size=0.25, random_state=42)

In [6]:
# the labels of the X_train
y_train = X_train['intrusion']

# dataset excluding target attribute (encoded, one-hot-encoded,original)
X_train = X_train.drop(['intrusion','Dos','normal','Probe','R2L','U2R','label'],axis=1)

# the labels of the X_test
y_test = X_test['intrusion']

# dataset excluding target attribute (encoded, one-hot-encoded,original)
X_test = X_test.drop(['intrusion','Dos','normal','Probe','R2L','U2R','label'],axis=1)

In [7]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

In [9]:
class DenseTranspose(keras.layers.Layer):
  def __init__(self, dense, activation=None, **kwargs):
    self.dense = dense
    self.activation = keras.activations.get(activation)
    super().__init__(**kwargs)
  def build(self, batch_input_shape):
    self.biases = self.add_weight(name="bias", initializer="zeros",shape=[self.dense.input_shape[-1]])
    self.W = tf.transpose(self.dense.weights[0])
    super().build(batch_input_shape)
  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.dense.input_shape[-1])
  def call(self, inputs):
    z = tf.matmul(inputs, self.W)
    return self.activation(z + self.biases)

In [15]:
keras.backend.clear_session()
num_hidden = (X_train.shape[1], 14, 28, 28)

Dense_11 = Dense(units=num_hidden[1], activation='relu')
Dense_12 = Dense(units=num_hidden[2], activation='relu')
Dense_13 = Dense(units=num_hidden[3], activation='relu')

inputs_1 = Input(shape=(num_hidden[0],))

#Encoder
encoded_11 = Dense_11(inputs_1)
encoded_12 = Dense_12(encoded_11)
encoded_13 = Dense_13(encoded_12)

#Decoder
decoded_11 = DenseTranspose(Dense_13, activation='relu')(encoded_13)
decoded_12 = DenseTranspose(Dense_12, activation='relu')(decoded_11)
outputs_1 = DenseTranspose(Dense_11, activation='relu')(decoded_12)

AE_1=Model(inputs_1, outputs_1)
Encoder_1=Model(inputs_1, decoded_12)

In [ ]:
AE_1.compile(optimizer='adam', loss= 'mean_squared_error', metrics=['accuracy'])
AE_1.fit(X_train,X_train,epochs=100,batch_size=256,shuffle=True)

In [18]:
AE_1_encoded_train = Encoder_1.predict(X_train)
AE_1_encoded_test = Encoder_1.predict(X_test)

985/985 [==============================] - 2s 2ms/step


In [22]:
keras.backend.clear_session()
num_hidden = (14, 28, 28)

Dense_21 = Dense(units=num_hidden[1], activation='relu')
Dense_22 = Dense(units=num_hidden[2], activation='relu')


inputs_2 = Input(shape=(num_hidden[0],))

#Encoder
encoded_21 = Dense_21(inputs_2)
encoded_22 = Dense_22(encoded_21)

#Decoder
decoded_21 = DenseTranspose(Dense_22, activation='relu')(encoded_22)
outputs_2 = DenseTranspose(Dense_21, activation='relu')(decoded_21)

AE_2=Model(inputs_2, outputs_2)
Encoder_2=Model(inputs_2, decoded_21)

In [23]:
AE_2.compile(optimizer='adam', loss= 'mean_squared_error', metrics=['accuracy'])
AE_2.fit(AE_1_encoded_train,AE_1_encoded_train,epochs=100,batch_size=256,shuffle=True)

Epoch 1/100
370/370 [==============================] - 2s 2ms/step - loss: 49.3694 - accuracy: 0.5141
Epoch 2/100
370/370 [==============================] - 1s 2ms/step - loss: 31.4041 - accuracy: 0.8673
Epoch 3/100
370/370 [==============================] - 1s 2ms/step - loss: 29.1017 - accuracy: 0.8772
Epoch 4/100
370/370 [==============================] - 1s 2ms/step - loss: 27.5419 - accuracy: 0.8984
Epoch 5/100
370/370 [==============================] - 1s 4ms/step - loss: 26.2957 - accuracy: 0.9229
Epoch 6/100
370/370 [==============================] - 2s 5ms/step - loss: 25.1568 - accuracy: 0.9656
Epoch 7/100
370/370 [==============================] - 1s 2ms/step - loss: 23.4909 - accuracy: 0.9846
Epoch 8/100
370/370 [==============================] - 1s 4ms/step - loss: 22.4932 - accuracy: 0.9839
Epoch 9/100
370/370 [==============================] - 1s 4ms/step - loss: 20.7212 - accuracy: 0.9875
Epoch 10/100
370/370 [==============================] - 1s 3ms/step - loss: 19.308

In [24]:
AE_2_encoded_train = Encoder_2.predict(AE_1_encoded_train)
AE_2_encoded_test = Encoder_2.predict(AE_1_encoded_test)

985/985 [==============================] - 2s 2ms/step


In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc1 = RandomForestClassifier(n_jobs=-1, n_estimators=10)
rfc1.fit(AE_2_encoded_train, y_train)
predictions = rfc1.predict(AE_2_encoded_test)
rfc1.score(AE_2_encoded_test, y_test)

0.9807264875849369

In [26]:
print(classification_report(y_test, predictions,target_names=le2_classes_))

              precision    recall  f1-score   support

         Dos       0.99      0.98      0.99     11484
       Probe       0.95      0.96      0.96      2947
         R2L       0.88      0.84      0.86       274
         U2R       0.33      0.20      0.25        15
      normal       0.98      0.99      0.98     16774

    accuracy                           0.98     31494
   macro avg       0.83      0.79      0.81     31494
weighted avg       0.98      0.98      0.98     31494

